# Creation of my first neural network

## Feed Forward Back Propagation

se creará una red neuronal feed forward back propagation con una capa de entrada, una capa oculta y una capa de salida

retorpropagacion: https://www.youtube.com/watch?v=tIeHLnjs5U8

backpropagation: https://towardsdatascience.com/understanding-backpropagation-algorithm-7bb3aa2f95fd


In [2]:
# importacion de librerias
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [3]:
# Carga de los datos de entrenamiento
d_train = pd.read_csv("dataset/dataset_train.csv")
d_train.head(5)
d_train.groupby('rain').describe()


id                                                         \
      count   mean        std    min     25%    50%     75%    max   
rain                                                                 
1      91.0   46.0  26.413380    1.0   23.50   46.0   68.50   91.0   
2      86.0  134.5  24.969982   92.0  113.25  134.5  155.75  177.0   
3     126.0  240.5  36.517119  178.0  209.25  240.5  271.75  303.0   

     temperature             ... humidity       radiation             \
           count       mean  ...      75%   max     count       mean   
rain                         ...                                       
1           91.0  16.485714  ...   88.050  94.4      91.0  50.332198   
2           86.0  18.137209  ...   70.725  79.8      86.0  47.877442   
3          126.0  23.950000  ...   44.575  49.4     126.0  75.013651   

                                                         
            std    min      25%     50%      75%    max  
rain                                                     
1      5.818797  40.56  44.8650  50.600  55.6150  59.31  
2     17.713754  20.09  32.4275  46.750  63.3775  78.73  
3      9.051908  60.40  66.1025  75.685  83.4850  89.95  

[3 rows x 32 columns]

In [4]:
# Carga de los datos de validacion
d_valid = pd.read_csv("dataset/dataset_val.csv")
d_valid.head(5)
d_valid.groupby('rain').describe()


id                                                 temperature  \
     count  mean       std   min    25%   50%    75%   max       count   
rain                                                                     
1     30.0  15.5  8.803408   1.0   8.25  15.5  22.75  30.0        30.0   
2     33.0  47.0  9.669540  31.0  39.00  47.0  55.00  63.0        33.0   
3     34.0  80.5  9.958246  64.0  72.25  80.5  88.75  97.0        34.0   

                 ... humidity       radiation                              \
           mean  ...      75%   max     count       mean        std   min   
rain             ...                                                        
1     16.253333  ...   83.450  92.2      30.0  49.766667   6.356063  40.3   
2     18.218182  ...   68.900  77.7      33.0  50.454545  16.091304  24.8   
3     24.705882  ...   45.875  50.0      34.0  73.711765   8.763449  60.1   

                                   
         25%    50%     75%   max  
rain                               
1     44.175  50.45  55.225  59.3  
2     35.300  51.70  58.600  80.0  
3     68.125  72.50  81.700  88.5  

[3 rows x 32 columns]

In [5]:
# Carga de los datos de test
d_test = pd.read_csv("dataset/dataset_test.csv")
d_test.head(5)
d_test.groupby('rain').describe()


id                                                 temperature  \
     count  mean       std   min    25%   50%    75%   max       count   
rain                                                                     
1     30.0  15.5  8.803408   1.0   8.25  15.5  22.75  30.0        30.0   
2     33.0  47.0  9.669540  31.0  39.00  47.0  55.00  63.0        33.0   
3     34.0  80.5  9.958246  64.0  72.25  80.5  88.75  97.0        34.0   

                 ... humidity       radiation                              \
           mean  ...      75%   max     count       mean        std   min   
rain             ...                                                        
1     16.660000  ...   85.650  94.6      30.0  50.256667   5.071603  41.6   
2     18.021212  ...   75.900  79.9      33.0  53.869697  16.922368  20.1   
3     24.155882  ...   46.775  49.6      34.0  74.926471   8.351095  60.9   

                                  
         25%   50%     75%   max  
rain                              
1     45.800  50.1  54.750  58.6  
2     37.900  55.8  64.400  78.9  
3     65.975  76.4  82.175  88.6  

[3 rows x 32 columns]

## Normalizacion de dataset


In [6]:
def normDataset(x, val):
    return x/val


In [7]:
# normalizacion
d_train['temp_n'] = d_train['temperature'].apply(lambda x: normDataset(x, 30))
d_train[['humi_n', 'radi_n']] = d_train[['humidity', 'radiation']].apply(
    lambda x: normDataset(x, 100))

d_valid['temp_n'] = d_valid['temperature'].apply(lambda x: normDataset(x, 30))
d_valid[['humi_n', 'radi_n']] = d_valid[['humidity', 'radiation']].apply(
    lambda x: normDataset(x, 100))

d_test['temp_n'] = d_test['temperature'].apply(lambda x: normDataset(x, 30))
d_test[['humi_n', 'radi_n']] = d_test[['humidity', 'radiation']].apply(
    lambda x: normDataset(x, 100))

d_train.head(5)


,id,temperature,humidity,radiation,rain,temp_n,humi_n,radi_n
0,1,18.2,83.3,42.24,1,0.606667,0.833,0.4224
1,2,16.6,80.5,41.68,1,0.553333,0.805,0.4168
2,3,16.7,73.1,57.92,1,0.556667,0.731,0.5792
3,4,15.1,74.6,52.27,1,0.503333,0.746,0.5227
4,5,17.7,67.8,47.81,1,0.590000,0.678,0.4781


In [8]:
def mapOutput(x):
    if (x == 1):
        return [1, 0, 0]
    elif (x == 2):
        return [0, 1, 0]
    else:
        return [0, 0, 1]


In [9]:
d_train['output'] = d_train['rain'].apply(mapOutput)
d_valid['output'] = d_valid['rain'].apply(mapOutput)
d_test['output'] = d_test['rain'].apply(mapOutput)
d_train.head()


,id,temperature,humidity,radiation,rain,temp_n,humi_n,radi_n,output
0,1,18.2,83.3,42.24,1,0.606667,0.833,0.4224,"[1, 0, 0]"
1,2,16.6,80.5,41.68,1,0.553333,0.805,0.4168,"[1, 0, 0]"
2,3,16.7,73.1,57.92,1,0.556667,0.731,0.5792,"[1, 0, 0]"
3,4,15.1,74.6,52.27,1,0.503333,0.746,0.5227,"[1, 0, 0]"
4,5,17.7,67.8,47.81,1,0.590000,0.678,0.4781,"[1, 0, 0]"


## Separacion de los datos

agrupar los datos de entrenamiento y de validacion, separar las entradas de las salida y convertir las salidas en un vector de la forma [0,0,0] a paritr del dato


In [10]:
x_train = d_train[['temp_n', 'humi_n', 'radi_n']].to_numpy()
y_train = np.array(d_train['output'].values.tolist())

x_test = d_valid[['temp_n', 'humi_n', 'radi_n']].to_numpy()
y_test = np.array(d_test['output'].values.tolist())

# print(x_train[1])
# print(y_train)

# np.dot(x_train[1],y_train[300])


## Network creation

creation of neural network class


In [52]:

class NeuralNetwork:
    def __init__(self, n_input, n_hidden, n_output, train_rate=0.1):
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.n_output = n_output
        self.train_rate = train_rate

        # instatiation of weights matrix and bias vector
        # matrix from inputs to hidden
        self.weights_ih = np.random.rand(self.n_input, self.n_hidden)-0.5
        # matrix from hidden to output
        self.weights_ho = np.random.rand(self.n_hidden, self.n_output)-0.5
        # matrix from inputs to hidden
        self.bias_ih = np.random.rand(1, self.n_hidden)-0.5
        # matrix from inputs to hidden
        self.bias_ho = np.random.rand(1, self.n_output)-0.5

    def forwardPropagation(self, _input):
        self.ih_val = np.dot(_input, self.weights_ih) + self.bias_ih
        self.ih_val_act = self.transfer(self.ih_val)
        self.ho_val = np.dot(self.ih_val_act, self.weights_ho) + self.bias_ho
        self.output = self.transfer(self.ho_val)

        return self.output

    def transfer(self, _activation):
        # use of sigmoid function
        return 1.0 / (1.0+np.exp(-1*_activation))

    def transferdif(self, _activation):
        # use of sigmoid function
        return 1.0 / (1.0+np.exp(-1*_activation))

    def backPropError(self, _input, _target, _output):
        # error function using MSE function
        self.cost = (_output - _target)**2  # cost function MSE
        error_o = (2*(_output - _target)) # total cost derivative
        difOut = ((_output)*(1-_output))

        self.delta_w2 = np.dot(self.ih_val_act.T,difOut*error_o)/len(_output)
        self.delta_b2 = np.sum(difOut*error_o,axis=0,keepdims = True)/len(_output)

        error_h = np.dot(self.weights_ho,difOut.T*error_o.T)

        self.delta_w1 = np.dot(_input.T,error_h.T)/len(_output)
        self.delta_b1 = np.sum(error_h.T,axis=0,keepdims=True)/len(_output)
        
        self.tcost = np.sum(self.cost,axis=0,keepdims=True)/len(_output)

        # print(self.tcost)

        return

    def train(self, input, output, epochs = 10000, learn_rate=0.5 ):
        
        for i in range(epochs):
            self.forwardPropagation(input)
            self.backPropError(input, output, self.output)
            self.weights_ih -= learn_rate*self.delta_w1
            self.weights_ho -= learn_rate*self.delta_w2
            self.bias_ih -= learn_rate*self.delta_b1
            self.bias_ho -= learn_rate*self.delta_b2

            if(np.all(np.less(self.tcost,np.array([0.01,0.01,0.01])))):
                print("termino = ", i)
                break

        return

nnbp = NeuralNetwork(3, 4, 3)

nnbp.train(x_train,y_train)


[[0.22850607 0.25774446 0.29662515]]
[[0.22374973 0.24540439 0.28630054]]
[[0.21996923 0.23600736 0.27767137]]
[[0.21695889 0.22883648 0.27056989]]
[[0.21456106 0.22333943 0.26480665]]
[[0.21265142 0.21910194 0.26018598]]
[[0.21113039 0.21581639 0.25651886]]
[[0.20991768 0.2132545  0.25363212]]
[[0.20894858 0.21124621 0.25137365]]
[[0.20817113 0.20966407 0.24961419]]
[[0.20754385 0.20841187 0.24824675]]
[[0.20703377 0.20741649 0.24718462]]
[[0.20661485 0.20662201 0.24635857]]
[[0.2062666  0.20598538 0.24571406]]
[[0.20597297 0.2054733  0.2452085 ]]
[[0.20572144 0.20505986 0.24480884]]
[[0.20550222 0.20472483 0.24448962]]
[[0.2053077  0.20445232 0.24423126]]
[[0.20513198 0.20422982 0.24401876]]
[[0.20497047 0.20404744 0.24384066]]
[[0.20481959 0.20389732 0.24368818]]
[[0.20467657 0.20377321 0.24355466]]
[[0.20453925 0.20367011 0.24343496]]
[[0.20440597 0.20358401 0.2433252 ]]
[[0.20427541 0.20351171 0.24322237]]
[[0.20414656 0.20345062 0.2431242 ]]
[[0.20401861 0.20339863 0.24302891]]
[

In [53]:
# Validation

y_pred = nnbp.forwardPropagation(x_test)

In [54]:
[y_pred,y_test]


[array([[5.26514837e-01, 4.75065397e-01, 6.35367470e-03],
        [5.26514837e-01, 4.75065397e-01, 6.35367470e-03],
        [5.26514837e-01, 4.75065397e-01, 6.35367470e-03],
        [5.26514837e-01, 4.75065397e-01, 6.35367470e-03],
        [5.26514837e-01, 4.75065397e-01, 6.35367470e-03],
        [5.26514837e-01, 4.75065397e-01, 6.35367470e-03],
        [5.26514837e-01, 4.75065397e-01, 6.35367470e-03],
        [5.26514837e-01, 4.75065397e-01, 6.35367470e-03],
        [5.26514837e-01, 4.75065397e-01, 6.35367470e-03],
        [5.26514837e-01, 4.75065397e-01, 6.35367470e-03],
        [5.26514837e-01, 4.75065397e-01, 6.35367470e-03],
        [5.26514837e-01, 4.75065397e-01, 6.35367470e-03],
        [5.26514837e-01, 4.75065397e-01, 6.35367470e-03],
        [5.26514837e-01, 4.75065397e-01, 6.35367470e-03],
        [5.26514837e-01, 4.75065397e-01, 6.35367470e-03],
        [5.26514837e-01, 4.75065397e-01, 6.35367470e-03],
        [5.26514837e-01, 4.75065397e-01, 6.35367470e-03],
        [5.265